In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import copy
import scipy.ndimage as ndimage
from PIL import Image, ImageOps

def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

data = []
label = []
labels = []
images = []

for i in range(10):
    for j in range(10):
        image = tf.keras.preprocessing.image.load_img('./image/0'+str(i)+str(j)+'.png')
        images.append(image)
        input_arr = keras.preprocessing.image.img_to_array(image)
        data.append(input_arr)

for k in range(100):
    label.append(0)
for m in range(100):
    label[m] = 1
    temp = copy.deepcopy(label)
    labels.append(temp)
    label[m] = 0 
    
datas = np.array(data)
datas = datas / 255.0
labels = np.array(labels)
labels = labels / 1.0  

test_labels = copy.deepcopy(labels)

In [52]:
model = tf.keras.Sequential([ tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer='l2', input_shape=(512, 512, 3)),
                              tf.keras.layers.MaxPooling2D((2, 2)),
                              tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer='l2'),
                              tf.keras.layers.MaxPooling2D((2,2)),
                              tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer='l2'),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(64, activation='relu', kernel_regularizer='l2'),
                              tf.keras.layers.Dense(100, activation='softmax')])

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 510, 510, 32)      896       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 124, 124, 64)      36928     
_________________________________________________________________
flatten_15 (Flatten)         (None, 984064)            0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)              

In [37]:
'''
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(100, activation='softmax'))

model.summary()
'''

"\nmodel = models.Sequential()\nmodel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))\nmodel.add(layers.Flatten())\nmodel.add(layers.Dense(64, activation='relu'))\nmodel.add(layers.Dense(100, activation='softmax'))\n\nmodel.summary()\n"

In [44]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

for i in range(5):
    resized_datas = copy.deepcopy(datas)

    for i in range(len(resized_datas)):
        resize = int(np.round(512*np.random.uniform(0.8,1,1)))
        temp = tf.image.resize(datas[i], [resize, resize])
        if resize >= 512:
            temp2 = tf.image.resize_with_crop_or_pad(temp, 512, 512)
            resized_datas[i] = temp2
        elif resize < 512:
            imgtemp = keras.preprocessing.image.array_to_img(temp)
            delta_w = 512 - resize
            delta_h = 512 - resize
            padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
            imgtemp2 = ImageOps.expand(imgtemp, padding, fill=(255,255,255))
            temp2 = keras.preprocessing.image.img_to_array(imgtemp2)
            resized_datas[i] = temp2

    def random_rotate_image(image):
        image = ndimage.rotate(image, np.random.uniform(-45, 45), reshape=False, mode='constant', cval=255.0)
        return image

    rot_res_datas = copy.deepcopy(datas)

    for i in range(len(resized_datas)):
        rot_res_datas[i] = random_rotate_image(resized_datas[i])
        
    conct = tf.concat([rot_res_datas, datas], axis=0)
    train_datas = np.array(conct)
    conctl = tf.concat([labels, labels], axis=0)
    train_labels = np.array(conctl)
    
    model.fit(train_datas, train_labels, epochs=10)

Train on 200 samples
Epoch 1/10
200/200 [==============================] - 19s 97ms/sample - loss: 9.5532 - accuracy: 0.0400
Epoch 2/10
200/200 [==============================] - 18s 92ms/sample - loss: 5.8196 - accuracy: 0.1400
Epoch 3/10
200/200 [==============================] - 18s 92ms/sample - loss: 4.2309 - accuracy: 0.5100
Epoch 4/10
200/200 [==============================] - 18s 92ms/sample - loss: 3.6992 - accuracy: 0.5200
Epoch 5/10
200/200 [==============================] - 18s 92ms/sample - loss: 3.6031 - accuracy: 0.6950
Epoch 6/10
200/200 [==============================] - 18s 92ms/sample - loss: 3.3941 - accuracy: 0.8300
Epoch 7/10
200/200 [==============================] - 18s 92ms/sample - loss: 3.9095 - accuracy: 0.8650
Epoch 8/10
200/200 [==============================] - 18s 92ms/sample - loss: 2.7448 - accuracy: 0.9250
Epoch 9/10
200/200 [==============================] - 18s 92ms/sample - loss: 2.0495 - accuracy: 0.9300
Epoch 10/10
200/200 [======================

In [49]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

resized_datas = copy.deepcopy(datas)

for i in range(len(resized_datas)):
    resize = int(np.round(512*np.random.uniform(0.8,1,1)))
    temp = tf.image.resize(datas[i], [resize, resize])
    if resize >= 512:
        temp2 = tf.image.resize_with_crop_or_pad(temp, 512, 512)
        resized_datas[i] = temp2
    elif resize < 512:
        imgtemp = keras.preprocessing.image.array_to_img(temp)
        delta_w = 512 - resize
        delta_h = 512 - resize
        padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
        imgtemp2 = ImageOps.expand(imgtemp, padding, fill=(255,255,255))
        temp2 = keras.preprocessing.image.img_to_array(imgtemp2)
        resized_datas[i] = temp2

def random_rotate_image(image):
    image = ndimage.rotate(image, np.random.uniform(-45, 45), reshape=False, mode='constant', cval=255.0)
    return image

rot_res_datas = copy.deepcopy(datas)

for i in range(len(resized_datas)):
    rot_res_datas[i] = random_rotate_image(resized_datas[i])
    
model.fit(rot_res_datas, labels, epochs=10)

Train on 100 samples
Epoch 1/10
100/100 [==============================] - 10s 98ms/sample - loss: 9.8553 - accuracy: 0.1000
Epoch 2/10
100/100 [==============================] - 9s 91ms/sample - loss: 2.8391 - accuracy: 0.6400
Epoch 3/10
100/100 [==============================] - 9s 91ms/sample - loss: 1.3645 - accuracy: 0.9300
Epoch 4/10
100/100 [==============================] - 9s 91ms/sample - loss: 1.3898 - accuracy: 0.9700
Epoch 5/10
100/100 [==============================] - 9s 92ms/sample - loss: 1.5950 - accuracy: 0.9800
Epoch 6/10
100/100 [==============================] - 9s 92ms/sample - loss: 1.5143 - accuracy: 1.0000
Epoch 7/10
100/100 [==============================] - 9s 91ms/sample - loss: 1.4908 - accuracy: 1.0000
Epoch 8/10
100/100 [==============================] - 9s 92ms/sample - loss: 1.4375 - accuracy: 1.0000
Epoch 9/10
100/100 [==============================] - 9s 91ms/sample - loss: 1.3647 - accuracy: 1.0000
Epoch 10/10
100/100 [==============================

In [ ]:
test_loss, test_acc = model.evaluate(datas, labels, verbose=2)
print(test_acc)

In [45]:
resized_datas = copy.deepcopy(datas)

for i in range(len(resized_datas)):
    resize = int(np.round(512*np.random.uniform(0.8,1,1)))
    temp = tf.image.resize(datas[i], [resize, resize])
    if resize >= 512:
        temp2 = tf.image.resize_with_crop_or_pad(temp, 512, 512)
        resized_datas[i] = temp2
    elif resize < 512:
        imgtemp = keras.preprocessing.image.array_to_img(temp)
        delta_w = 512 - resize
        delta_h = 512 - resize
        padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
        imgtemp2 = ImageOps.expand(imgtemp, padding, fill=(255,255,255))
        temp2 = keras.preprocessing.image.img_to_array(imgtemp2)
        resized_datas[i] = temp2

def random_rotate_image(image):
    image = ndimage.rotate(image, np.random.uniform(-45, 45), reshape=False, mode='constant', cval=255.0)
    return image

rot_res_datas = copy.deepcopy(datas)

for i in range(len(resized_datas)):
    rot_res_datas[i] = random_rotate_image(resized_datas[i])
        
conct2 = tf.concat([rot_res_datas, rot_res_datas], axis=0)
test_datas = np.array(conct2)
conctl = tf.concat([labels, labels], axis=0)
test_labels = np.array(conctl)

test_loss, test_acc = model.evaluate(test_datas, test_labels, verbose=2)
print(test_acc)

200/1 - 3s - loss: 8.4395 - accuracy: 0.1000
0.1
